## Lift Chart Visualization 

This notebook shows you how to visualize the lift chart in Python. 

In [1]:
import sklearn.metrics
import pandas as pd 

Creating the cumulative gains calculation function 

In [4]:
def calc_cumulative_gains(df: pd.DataFrame, actual_col: str, predicted_col: str, probability_col: str):
    
    df.sort_values(by=probability_col, ascending= False, implace= True)
    subset = df[df[predicted_col] == True]
    
    rows = []
    for group in np.array_split(subset, 10):
        score = sklearn.metrics.accuracy_score(group[actual_col].tolist(), group[predicted_col].tolist(), normalize=False)
        rows.append({'NumCases': len(group), 'NumCorrectPredictions': score})
    
    lift = pd.DataFrame(rows)
    # Doing the Calculation of Cumulative Gains within the Dataframe
    
    lift['RunningCorrect'] = lift['NumCorrectPredictions'].cumsum()
    lift['PercentCorrect'] = lift.apply(
        lambda x: (100 / lift['NumCorrectPredictions'].sum())* x['RunningCorrect'], axis=1)
    lift['CumulativeCorrectBaseCase'] = lift['NumCases'].cumsum()
    lift['PercentCorrectBestCase'] = lift['CumulativeCorrectBaseCase'].apply(
        lambda x: 100 if (100 / lift['NumCorrectPredictions'].sum()) * x > 100 
                      else (100 / lift['NumCorrectPredictions'].sum()) * x)
    lift['AvgCase'] = lift['NumCorrectPredictions'].sum() / len(lift)
    lift['CumulativeAvgCase'] = lift['AvgCase'].cumsum()
    lift['PercentAvgCase'] = lift['CumulativeAvgCase'].apply(
        lambda x: (100 / lift['NumCorrectPredictions'].sum()) * x)
    
    # Constructing the Lift Chart 
    
    lift['NormalisedPercentAvg'] = 1
    lift['NormalisedPercentWithModel'] = lift['PercentCorrect'] / lift['PercentAvgCase']
    
    return lift 

Plotting Function for Lift Chart in Matplotlib 

In [5]:
import matplotlib.pyplot as plt 


In [6]:
def plot_cumulative_gains(lift: pd.DataFrame):
    fig, ax = plt.subplots()
    fig.canvas.draw()
    
    handles = []
    handles.append(ax.plot(lift['PercentCorrect'], 'r-', label='Percent Correct Predictions'))
    handles.append(ax.plot(lift['PercentCorrectBestCase'], 'g-', label='Best Case (for current model)'))
    handles.append(ax.plot(lift['PercentAvgCase'], 'b-', label='Average Case (for current model)'))
    ax.set_xlabel('Total Population (%)')
    ax.set_ylabel('Number of Respondents (%)')
    
    ax.set_xlim([0,9])
    ax.set_ylim([10, 100])
    
    labels = [int((label+1)*10) for label in [float(item.get_text()) for item in ax.get_xticklabels()]]
    ax.set_xticklabels(labels)
    
    fig.legend(handles, labels=[h[0].get_label() for h in handles])
    fig.show()

Function to visualize the lift chart 

In [ ]:
def plot_lift_chart(lift: pd.DataFrame):
    plt.figure()
    plt.plot(lift['NormalisedPercentAvg'], 'r-', label='Normalised \'response rate\'with no model')
    plt.plot(lift['NormalisedPercentWithModel'], 'g-', label='Normalised \'response rate\')